In [ ]:
### EXAMPLE 1

import pafy

url = "https://www.youtube.com/watch?v=1-qxab26dBc"
video = pafy.new(url)

streams = video.streams

for i in streams:
    print(i)

# get best resolution regardless of format
best = video.getbest()
  
print(best.resolution, best.extension)
  
# Download the video
best.download()

In [ ]:
### EXAMPLE 2
# pafy + vlc 選擇影片格式並下載

### IMPORT LIBRARY
import pafy
import vlc
import time
from moviepy.video.io.VideoFileClip import VideoFileClip
import urllib.request


### DEFINE FUNCTION
### 取得terminal視窗大小
def get_cmd_size():
    import shutil
    col, line = shutil.get_terminal_size()
    return col, line


### 打印分隔用
def print_div(text=None):
    col,line = get_cmd_size()
    col = col-1
    if text == None:
        print('-'*col)
    else:
        print('-'*col)
        print(text)
        print('-'*col)


### 計時、進度條
def clock(sec, sign = '#'):
    col, line = get_cmd_size()
    col_ = col - 42
    bar_bg = ' '*(col_)
    print_div()

    for i in range(sec+1):
        
        bar_idx = (col_//sec*(i+1))
        bar = ''
        for t in range(col_): bar += sign if t <= bar_idx else bar_bg[t]
        
        percent = int(100/sec*(i))
        end = '\n' if i==sec else '\r'
        print('Download Stream Video [{:02}/{:02}s] [{}] ({:02}%)'.format(i, sec, bar, percent), end=end)
        time.sleep(0.5)


### 確認url是否有效
def check_url(url):
    import urllib.request
    code = urllib.request.urlopen(url).getcode()
    if str(code).startswith('2') or str(code).startswith('3'):
        print('影片連結正常!')
    else:
        print('影片連結失效!')


### 下載影片並檢查是否下載完成
def play_video(url, save_name):
    # creating a vlc instance
    vlc_instance = vlc.Instance()

    # creating a media player
    player = vlc_instance.media_player_new()

    # creating a media
    media = vlc_instance.media_new(url)

    # 命名欲儲存之影片
    media.get_mrl()
    media.add_option(f"sout=file/ts:{save_name}") 

    # setting media to the player
    player.set_media(media)

    # play the video
    player.play()

    # 確認影片為下載階段或下載完畢
    t = 0
    sign = '.'
    end = '\r'
    good_states = ["State.Playing", "State.NothingSpecial", "State.Opening"]
    
    while str(player.get_state()) in good_states:
#         print('Stream is working. Current state = {}'.format(player.get_state()))
        bar = t*sign + ' '*(3-t)
        print('Download{}'.format(bar), end=end)
        t += 1
        if t > 3:
            t = 0
        time.sleep(0.5)

    print('Stream is not working. Current state = {}'.format(player.get_state()))
    
    # 釋放資源
    player.stop()


### 擷取特定秒數並儲存
def cut_video(video_name, sec, save_name):
    print_div()
    print('Cutting Video Used Moviepy\n')
    time.sleep(1)
    
    with VideoFileClip(video_name) as video:
        new = video.subclip(0, sec)
        new.write_videofile(save_name)
   
    print_div(f'save file {save_name}')



### MAIN
# 欲儲存影片之名稱及youtube來源
video_name = 'org.mp4'
save_name = 'cut.mp4'
sec = 10
url = "https://www.youtube.com/watch?v=CUfFgRcKzAg"
video = pafy.new(url)

# 確認影片連結是否正確
check_url(url)

# 取得串流來源列表
streams = video.streams
for i, stream in enumerate(streams):
    print('{}. {}'.format(i+1, stream))

idx = int(input('請選擇影片格式之序號:'))
while idx not in range(1, i+2):
    idx = int(input('超出範圍!!請重新選擇影片格式之序號:'))

target_video = streams[idx-1] 
print('您選擇的影片格式是: {}'.format(target_video))
print('影片URL: {}'.format(target_video.url))

# 下載影片
play_video(target_video.url, video_name)

### 進度條、擷取影片
# 播放 sec 秒 同時運行 進度條
# clock(sec)

# 裁切影片，因為停n秒長度不一定是n
cut_video(video_name, sec, save_name)